In [1]:
# load library
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
import pickle
import zstandard as zstd
from utils import read_zst
from sklearn.preprocessing import MinMaxScaler
import os
from LSTMModel import LSTMModel


### Data Preprocessing

In [3]:
# normalize 'AGE' between 0 and 1
scaler = MinMaxScaler()
train_X = read_zst('./stored_files/train_x.zst')
train_X['AGE'] = scaler.fit_transform(train_X[['AGE']])
pickle.dump(scaler, open('./stored_files/age_scaler.pkl', 'wb'))
train_y = read_zst('./stored_files/train_y.zst')

### LSTM Model

In [4]:
input_size = 518  # number of features
hidden_size = 64
output_size = 49  # number of unique diagnoses
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = LSTMModel(input_size, hidden_size, output_size).to(device)

In [5]:
# convert arrays to PyTorch tensors and move them to the specified device
X_train_tensor = torch.tensor(train_X.values, dtype=torch.float).to(device)
y_train_tensor = torch.tensor(train_y.values, dtype=torch.long).to(device)
# create TensorDataset and DataLoader for batch processing
train_data = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(dataset=train_data, batch_size=64, shuffle=True)
# define the loss function and optimizer
criterion = torch.nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [33]:
# set the number of eposhes
num_epochs = 100

In [34]:
# apply the model to the training loop
best_loss = float('inf')
for epoch in range(num_epochs):
    total_loss = 0
    num_batches = 0
    
    for i, (features, labels) in enumerate(train_loader):
        # assuming labels are initially Long for indices, convert them to float for BCEWithLogitsLoss
        labels = labels.float()  # convert labels to float

        features = features.unsqueeze(1) # unsqueeze the data's feature to fit the model

        outputs = model(features)
        loss = criterion(outputs, labels) 
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
        num_batches += 1

    # Calculate average loss for the epoch
    avg_loss = total_loss / num_batches
    print(f'Epoch [{epoch + 1}/{num_epochs}], Average Loss: {avg_loss:.4f}')

    # Save the model if the average loss of this epoch is the lowest encountered so far
    if avg_loss < best_loss:
        best_loss = avg_loss
        # Save the model checkpoint
        torch.save(model.state_dict(), 'best_model.pth')
        print(f"Saved better model with Average Loss: {best_loss:.4f}")

Epoch [1/100], Average Loss: 0.1056
Saved better model with Average Loss: 0.1056
Epoch [2/100], Average Loss: 0.0587
Saved better model with Average Loss: 0.0587
Epoch [3/100], Average Loss: 0.0515
Saved better model with Average Loss: 0.0515
Epoch [4/100], Average Loss: 0.0481
Saved better model with Average Loss: 0.0481
Epoch [5/100], Average Loss: 0.0459
Saved better model with Average Loss: 0.0459
Epoch [6/100], Average Loss: 0.0444
Saved better model with Average Loss: 0.0444
Epoch [7/100], Average Loss: 0.0433
Saved better model with Average Loss: 0.0433
Epoch [8/100], Average Loss: 0.0424
Saved better model with Average Loss: 0.0424
Epoch [9/100], Average Loss: 0.0417
Saved better model with Average Loss: 0.0417
Epoch [10/100], Average Loss: 0.0410
Saved better model with Average Loss: 0.0410
Epoch [11/100], Average Loss: 0.0405
Saved better model with Average Loss: 0.0405
Epoch [12/100], Average Loss: 0.0401
Saved better model with Average Loss: 0.0401
Epoch [13/100], Average L

### Apply the model to test dataset

In [35]:
model.load_state_dict(torch.load('./stored_files/best_model.pth'))

<All keys matched successfully>

In [36]:
test_X = read_zst('./stored_files/test_x.zst')
test_X['AGE'] = scaler.transform(test_X[['AGE']])
test_y = read_zst('./stored_files/test_y.zst')

In [37]:
# convert test data to PyTorch tensors and move them to the device
X_test_tensor = torch.tensor(test_X.values, dtype=torch.float).to(device)
y_test_tensor = torch.tensor(test_y.values, dtype=torch.long).to(device)  # If your task is classification

In [38]:
# apply the model to test dataset
model.eval()  # set the model to evaluation mode
all_predictions = []
all_labels = []
with torch.no_grad():
    for features in DataLoader(X_test_tensor, batch_size=32):
        features = features.unsqueeze(1) 
        outputs = model(features)
        predictions = torch.sigmoid(outputs).round()
        all_predictions.extend(predictions.cpu().numpy())

all_labels = y_test_tensor.cpu().numpy()

### Evaluate the model

In [41]:
from metric_utils import compute_metric
# apply the function to get the compute metric of the model
metrics = compute_metric(all_labels, np.array(all_predictions))
print(metrics)

{'ACC': 0.997, 'DDR': 0.8920586636340337, 'DDP': 0.888055405722262, 'DDF1': 0.8862359827661974, 'GM': 0.9243636922721007}


In [42]:
import json
with open("metrics.json", "w") as outfile: 
    json.dump(metrics, outfile)